In [3]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import Adam
import matplotlib.pyplot as plt

# Set the number of points for point cloud sampling/padding
NUM_POINTS = 2048

##################
# ModelNet40 Dataset Class
##################
class ModelNet40Dataset(Dataset):
    def __init__(self, root_dir, classes, split='train', normalization="shape_unit", target_num_points=2048):
        self.root_dir = root_dir
        self.classes = classes
        self.split = split  # 'train' or 'test'
        self.normalization = normalization
        self.target_num_points = target_num_points  # Zielanzahl der Punkte
        self.data, self.labels = self.load_data()

    def load_data(self):
        data, labels = [], []
        for class_idx, class_name in enumerate(self.classes):
            class_dir = os.path.join(self.root_dir, class_name, self.split)  # Navigate to the 'train' or 'test' folder
            if not os.path.exists(class_dir):
                continue  # Skip if the folder doesn't exist
            for file in os.listdir(class_dir):
                if file.endswith(".off"):
                    filepath = os.path.join(class_dir, file)
                    points = self.load_off_file(filepath)
                    data.append(points)
                    labels.append(class_idx)
        return data, labels

    @staticmethod
    def load_off_file(filepath):
        with open(filepath, "r") as f:
            lines = f.readlines()
        if lines[0].strip() == "OFF":
            lines = lines[1:]
        else:
            lines[0] = lines[0][3:]  # Handle case where 'OFF' is not at the start
        n_vertices, _, _ = map(int, lines[0].split())
        points = np.array([list(map(float, line.split())) for line in lines[1:1 + n_vertices]])
        return points

    def normalize(self, points):
        if self.normalization == "shape_unit":
            centroid = np.mean(points, axis=0)
            points -= centroid
            max_distance = np.max(np.linalg.norm(points, axis=1))
            points /= max_distance
        elif self.normalization == "shape_bbox":
            bbox_min = np.min(points, axis=0)
            bbox_max = np.max(points, axis=0)
            points = (points - bbox_min) / (bbox_max - bbox_min)
        return points

    def downsample_points(self, points):
        num_points = points.shape[0]
        if num_points > self.target_num_points:
            indices = np.random.choice(num_points, self.target_num_points, replace=False)
            points = points[indices]
        elif num_points < self.target_num_points:
            padding = self.target_num_points - num_points
            points = np.vstack([points, np.zeros((padding, 3))])
        return points

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        points = self.data[idx]
        label = self.labels[idx]

        # Normalisierung anwenden
        points = self.normalize(points)

        # Downsampling/Padding anwenden
        points = self.downsample_points(points)

        return torch.tensor(points, dtype=torch.float32), label

##################
# Checkpoint Manager
##################
class CheckpointManager:
    def __init__(self, save_dir):
        os.makedirs(save_dir, exist_ok=True)
        self.save_dir = save_dir

    def save(self, model, optimizer, epoch, loss):
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch,
            'loss': loss,
        }
        path = os.path.join(self.save_dir, f"checkpoint_epoch_{epoch}.pt")
        torch.save(checkpoint, path)

    def load(self, model, optimizer, checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        return checkpoint['epoch'], checkpoint['loss']

##################
# Common Code
##################
def reparameterize_gaussian(mean, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn(std.size()).to(mean.device)
    return mean + std * eps


##################
# PointNet Encoder
##################
class PointNetEncoder(nn.Module):
    def __init__(self, zdim, input_dim=3):
        super().__init__()
        self.zdim = zdim
        self.conv1 = nn.Conv1d(input_dim, 128, 1)
        self.conv2 = nn.Conv1d(128, 128, 1)
        self.conv3 = nn.Conv1d(128, 256, 1)
        self.conv4 = nn.Conv1d(256, 512, 1)
        self.bn1 = nn.BatchNorm1d(128)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(512)

        self.fc1_m = nn.Linear(512, 256)
        self.fc2_m = nn.Linear(256, 128)
        self.fc3_m = nn.Linear(128, zdim)
        self.fc_bn1_m = nn.BatchNorm1d(256)
        self.fc_bn2_m = nn.BatchNorm1d(128)

        self.fc1_v = nn.Linear(512, 256)
        self.fc2_v = nn.Linear(256, 128)
        self.fc3_v = nn.Linear(128, zdim)
        self.fc_bn1_v = nn.BatchNorm1d(256)
        self.fc_bn2_v = nn.BatchNorm1d(128)

    def forward(self, x):
        x = x.transpose(1, 2)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.bn4(self.conv4(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 512)

        m = F.relu(self.fc_bn1_m(self.fc1_m(x)))
        m = F.relu(self.fc_bn2_m(self.fc2_m(m)))
        m = self.fc3_m(m)
        v = F.relu(self.fc_bn1_v(self.fc1_v(x)))
        v = F.relu(self.fc_bn2_v(self.fc2_v(v)))
        v = self.fc3_v(v)
        return m, v
    
##################
# Autoencoder
##################
class AutoEncoder(nn.Module):
    def __init__(self, args):
        super(AutoEncoder, self).__init__()
        self.args = args
        self.encoder = PointNetEncoder(zdim=self.args["latent_dim"])
        self.diffusion = DiffusionPoint(
            net=PointwiseNet(point_dim=3, context_dim=self.args["latent_dim"], residual=self.args["residual"]),
            var_sched=VarianceSchedule(
                num_steps=self.args["num_steps"],
                beta_1=self.args["beta_1"],
                beta_T=self.args["beta_T"],
                mode=self.args["sched_mode"]
            )
        )


    def encode(self, x):
        code, _ = self.encoder(x)
        return code

    def decode(self, code, num_points, flexibility=0.0, ret_traj=False):
        return self.diffusion.sample(num_points, code, flexibility=flexibility, ret_traj=ret_traj)

    def get_loss(self, x):
        code = self.encode(x)
        loss = self.diffusion.get_loss(x, code)
        return loss

##################
# Diffusion Components
##################
class VarianceSchedule(nn.Module):

    def __init__(self, num_steps, beta_1, beta_T, mode='linear'):
        super().__init__()
        assert mode in ('linear', )
        self.num_steps = num_steps
        self.beta_1 = beta_1
        self.beta_T = beta_T
        self.mode = mode

        if mode == 'linear':
            betas = torch.linspace(beta_1, beta_T, steps=num_steps)

        betas = torch.cat([torch.zeros([1]), betas], dim=0)     # Padding

        alphas = 1 - betas
        log_alphas = torch.log(alphas)
        for i in range(1, log_alphas.size(0)):  # 1 to T
            log_alphas[i] += log_alphas[i - 1]
        alpha_bars = log_alphas.exp()

        sigmas_flex = torch.sqrt(betas)
        sigmas_inflex = torch.zeros_like(sigmas_flex)
        for i in range(1, sigmas_flex.size(0)):
            sigmas_inflex[i] = ((1 - alpha_bars[i-1]) / (1 - alpha_bars[i])) * betas[i]
        sigmas_inflex = torch.sqrt(sigmas_inflex)

        self.register_buffer('betas', betas)
        self.register_buffer('alphas', alphas)
        self.register_buffer('alpha_bars', alpha_bars)
        self.register_buffer('sigmas_flex', sigmas_flex)
        self.register_buffer('sigmas_inflex', sigmas_inflex)

    def uniform_sample_t(self, batch_size):
        ts = np.random.choice(np.arange(1, self.num_steps+1), batch_size)
        return ts.tolist()

    def get_sigmas(self, t, flexibility):
        assert 0 <= flexibility and flexibility <= 1
        sigmas = self.sigmas_flex[t] * flexibility + self.sigmas_inflex[t] * (1 - flexibility)
        return sigmas


class PointwiseNet(nn.Module):

    def __init__(self, point_dim, context_dim, residual):
        super().__init__()
        self.act = F.leaky_relu
        self.residual = residual
        self.layers = nn.ModuleList([
            ConcatSquashLinear(3, 128, context_dim+3),
            ConcatSquashLinear(128, 256, context_dim+3),
            ConcatSquashLinear(256, 512, context_dim+3),
            ConcatSquashLinear(512, 256, context_dim+3),
            ConcatSquashLinear(256, 128, context_dim+3),
            ConcatSquashLinear(128, 3, context_dim+3)
        ])

    def forward(self, x, beta, context):
        batch_size = x.size(0)
        beta = beta.view(batch_size, 1, 1)        
        context = context.view(batch_size, 1, -1)  

        time_emb = torch.cat([beta, torch.sin(beta), torch.cos(beta)], dim=-1) 
        ctx_emb = torch.cat([time_emb, context], dim=-1)  

        out = x
        for i, layer in enumerate(self.layers):
            out = layer(ctx=ctx_emb, x=out)
            if i < len(self.layers) - 1:
                out = self.act(out)

        if self.residual:
            return x + out
        else:
            return out


class DiffusionPoint(nn.Module):

    def __init__(self, net, var_sched:VarianceSchedule):
        super().__init__()
        self.net = net
        self.var_sched = var_sched

    def get_loss(self, x_0, context, t=None):
        batch_size, _, point_dim = x_0.size()
        if t == None:
            t = self.var_sched.uniform_sample_t(batch_size)
        alpha_bar = self.var_sched.alpha_bars[t]
        beta = self.var_sched.betas[t]

        c0 = torch.sqrt(alpha_bar).view(-1, 1, 1)       # (B, 1, 1)
        c1 = torch.sqrt(1 - alpha_bar).view(-1, 1, 1)   # (B, 1, 1)

        e_rand = torch.randn_like(x_0)  # (B, N, d)
        e_theta = self.net(c0 * x_0 + c1 * e_rand, beta=beta, context=context)

        loss = F.mse_loss(e_theta.view(-1, point_dim), e_rand.view(-1, point_dim), reduction='mean')
        return loss

    def sample(self, num_points, context, point_dim=3, flexibility=0.0, ret_traj=False):
        batch_size = context.size(0)
        x_T = torch.randn([batch_size, num_points, point_dim]).to(context.device)
        traj = {self.var_sched.num_steps: x_T}

        for t in range(self.var_sched.num_steps, 0, -1):
            z = torch.randn_like(x_T) if t > 1 else torch.zeros_like(x_T)
            alpha = self.var_sched.alphas[t]
            alpha_bar = self.var_sched.alpha_bars[t]
            sigma = self.var_sched.get_sigmas(t, flexibility)

            c0 = 1.0 / torch.sqrt(alpha)
            c1 = (1 - alpha) / torch.sqrt(1 - alpha_bar)

            x_t = traj[t]
            beta = self.var_sched.betas[[t]*batch_size]
            e_theta = self.net(x_t, beta=beta, context=context)
            x_next = c0 * (x_t - c1 * e_theta) + sigma * z
            traj[t-1] = x_next.detach()     
            traj[t] = traj[t].cpu()      

            if not ret_traj:
                del traj[t]         
        
        if ret_traj:
            return traj
        else:
            return traj[0]
        
class ConcatSquashLinear(nn.Module):
    def __init__(self, dim_in, dim_out, dim_ctx):
        super(ConcatSquashLinear, self).__init__()
        self._layer = nn.Linear(dim_in, dim_out)
        self._hyper_bias = nn.Linear(dim_ctx, dim_out, bias=False)
        self._hyper_gate = nn.Linear(dim_ctx, dim_out)

    def forward(self, ctx, x):
        gate = torch.sigmoid(self._hyper_gate(ctx))
        bias = self._hyper_bias(ctx)
        # if x.dim() == 3:
        #     gate = gate.unsqueeze(1)
        #     bias = bias.unsqueeze(1)
        ret = self._layer(x) * gate + bias
        return ret

##################
# Visualization
##################
def visualize_point_cloud(point_cloud, title="Generated Point Cloud"):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection="3d")
    x, y, z = point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2]
    ax.scatter(x, y, z, c=z, cmap="viridis", s=1)
    ax.set_title(title)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    plt.show()

import open3d as o3d

def visualize_with_open3d(points, title="Point Cloud Visualization"):
    """
    Visualize point cloud using Open3D.
    
    Args:
        points: numpy array of shape (N, 3)
        title: window title
    """
    # Create Open3D point cloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    # Optional: add color (blue)
    pcd.paint_uniform_color([0, 0, 1])
    
    # Visualize
    o3d.visualization.draw_geometries([pcd], window_name=title)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
import os
import torch
import tqdm
import numpy as np
import random
import time
import logging
import logging.handlers
from scipy.linalg import sqrtm

THOUSAND = 1000
MILLION = 1000000


class BlackHole(object):
    def __setattr__(self, name, value):
        pass
    def __call__(self, *args, **kwargs):
        return self
    def __getattr__(self, name):
        return self


class CheckpointManager(object):

    def __init__(self, save_dir, logger=BlackHole()):
        super().__init__()
        os.makedirs(save_dir, exist_ok=True)
        self.save_dir = save_dir
        self.ckpts = []
        self.logger = logger

        for f in os.listdir(self.save_dir):
            if f[:4] != 'ckpt':
                continue
            _, score, it = f.split('_')
            it = it.split('.')[0]
            self.ckpts.append({
                'score': float(score),
                'file': f,
                'iteration': int(it),
            })     

    def save(self, model, args, score, others=None, step=None):

        if step is None:
            fname = 'ckpt_%.6f_.pt' % float(score)
        else:
            fname = 'ckpt_%.6f_%d.pt' % (float(score), int(step))
        path = os.path.join(self.save_dir, fname)

        torch.save({
            'args': args,
            'state_dict': model.state_dict(),
            'others': others
        }, path)

        self.ckpts.append({
            'score': score,
            'file': fname
        })

        return True

def seed_all(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


def get_logger(name, log_dir=None):
    logger = logging.getLogger(name)
    
    # Verhindere doppelte Handler
    if not logger.hasHandlers():
        logger.setLevel(logging.DEBUG)
        formatter = logging.Formatter('[%(asctime)s::%(name)s::%(levelname)s] %(message)s')

        stream_handler = logging.StreamHandler()
        stream_handler.setLevel(logging.DEBUG)
        stream_handler.setFormatter(formatter)
        logger.addHandler(stream_handler)

        if log_dir is not None:
            file_handler = logging.FileHandler(os.path.join(log_dir, 'log.txt'))
            file_handler.setLevel(logging.INFO)
            file_handler.setFormatter(formatter)
            logger.addHandler(file_handler)

    return logger

#Evaluation

def distChamfer(a, b):
    x, y = a, b
    bs, num_points, points_dim = x.size()
    xx = torch.bmm(x, x.transpose(2, 1))
    yy = torch.bmm(y, y.transpose(2, 1))
    zz = torch.bmm(x, y.transpose(2, 1))
    diag_ind = torch.arange(0, num_points).to(a).long()
    rx = xx[:, diag_ind, diag_ind].unsqueeze(1).expand_as(xx)
    ry = yy[:, diag_ind, diag_ind].unsqueeze(1).expand_as(yy)
    P = (rx.transpose(2, 1) + ry - 2 * zz)
    return P.min(1)[0], P.min(2)[0]

def calculate_fid(features_real, features_fake):
    mu_real, sigma_real = np.mean(features_real, axis=0), np.cov(features_real, rowvar=False)
    mu_fake, sigma_fake = np.mean(features_fake, axis=0), np.cov(features_fake, rowvar=False)

    # Compute squared difference of means
    mean_diff = np.sum((mu_real - mu_fake) ** 2)

    # Compute sqrt of product of covariance matrices
    cov_sqrt = sqrtm(sigma_real @ sigma_fake)

    # Handle numerical instability
    if np.iscomplexobj(cov_sqrt):
        cov_sqrt = cov_sqrt.real

    fid = mean_diff + np.trace(sigma_real + sigma_fake - 2 * cov_sqrt)
    return fid

def chamfer(sample_pcs, ref_pcs, batch_size, reduced=True):
    N_sample = sample_pcs.shape[0]
    N_ref = ref_pcs.shape[0]
    assert N_sample == N_ref, "REF:%d SMP:%d" % (N_ref, N_sample)

    cd_lst = []
    emd_lst = []
    iterator = range(0, N_sample, batch_size)

    for b_start in tqdm(iterator, desc='EMD-CD'):
        b_end = min(N_sample, b_start + batch_size)
        sample_batch = sample_pcs[b_start:b_end]
        ref_batch = ref_pcs[b_start:b_end]

        dl, dr = distChamfer(sample_batch, ref_batch)
        cd_lst.append(dl.mean(dim=1) + dr.mean(dim=1))

    if reduced:
        cd = torch.cat(cd_lst).mean()
    else:
        cd = torch.cat(cd_lst)

    results = {
        'MMD-CD': cd
    }
    return results

In [2]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.4.0+cu121
CUDA available: True
CUDA version: 12.1
Device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 3070


In [5]:
# AE Training

import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import os
import time

# Arguments
args = {
    'latent_dim': 256,
    'num_steps': 200,
    'lr': 1e-3,
    'train_batch_size': 128,
    'val_batch_size': 32,
    'categories': ['airplane', 'chair', 'door', 'piano', 'stool', 'toilet', 'vase', 'tent', 'sofa', 'plant'],  # Category to test
    'device': 'cuda',  # Keep only this entry
    'residual': True,
    'beta_1': 1e-4,
    'beta_T': 0.05,
    'sched_mode': 'linear',
    'max_iters': 16000,
    'val_freq': 8000,
    'log_root': './logs_ae',
    'resume': './AE_all.pt' #./logs_ae/ckpt_0.183902_6000.pt
}


# Setup
seed_all(2020)
logger = get_logger('train', args['log_root'])
writer = SummaryWriter(args['log_root'])
ckpt_mgr = CheckpointManager(args['log_root'])

# Datasets and loaders
train_dset = ModelNet40Dataset(root_dir="Dataset/ModelNet40", classes=args['categories'], split='train', normalization="shape_unit", target_num_points=NUM_POINTS)
val_dset = ModelNet40Dataset(root_dir="Dataset/ModelNet40", classes=args['categories'], split='test', normalization="shape_unit", target_num_points=NUM_POINTS)

train_iter = torch.utils.data.DataLoader(train_dset, batch_size=args['train_batch_size'], shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dset, batch_size=args['val_batch_size'])

# Model
if args['resume'] is not None:
    ckpt = torch.load(args['resume'], map_location=torch.device('cuda'))
    
    # Ensure args compatibility
    if isinstance(ckpt['args'], dict):
        checkpoint_args = ckpt['args']
    else:  # If Namespace, convert to a dictionary
        checkpoint_args = vars(ckpt['args'])
    
    model = AutoEncoder(checkpoint_args).to(args['device'])
    model.load_state_dict(ckpt['state_dict'])
else:
    model = AutoEncoder(args).to(args['device'])

optimizer = optim.Adam(model.parameters(), lr=args['lr'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# Training loop
def train_step(batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0].to(args['device'])  # Get pointcloud
    loss = model.get_loss(x)
    loss.backward()
    optimizer.step()
    return loss

def validate():
    model.eval()
    total_loss = 0
    for batch in val_loader:
        with torch.no_grad():
            x = batch[0].to(args['device'])
            loss = model.get_loss(x)
            total_loss += loss.item()
    return total_loss / len(val_loader)

# Main loop
for it in range(1, args['max_iters'] + 1):
    batch = next(iter(train_iter))
    loss = train_step(batch)

    print(f"Step {it}/{args['max_iters']} - Train Loss: {loss.item()}")
    
    if it % args['val_freq'] == 0:
        val_loss = validate()
        writer.add_scalar('train/loss', loss.item(), it)
        writer.add_scalar('val/loss', val_loss, it)
        logger.info(f"Iteration {it} - Train Loss: {loss.item()}, Val Loss: {val_loss}")

        # Save checkpoint
        ckpt_mgr.save(model, args, val_loss, optimizer.state_dict(), step=it)

    scheduler.step()

C:\Users\Sirius\AppData\Local\Temp\ipykernel_19860\3131144993.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(args['resume'], map_location=torch.devic

Step 1/16000 - Train Loss: 0.3268086314201355
Step 2/16000 - Train Loss: 0.2155117690563202
Step 3/16000 - Train Loss: 0.21401028335094452
Step 4/16000 - Train Loss: 0.21532385051250458
Step 5/16000 - Train Loss: 0.24618244171142578
Step 6/16000 - Train Loss: 0.17763471603393555
Step 7/16000 - Train Loss: 0.15275143086910248
Step 8/16000 - Train Loss: 0.18907402455806732
Step 9/16000 - Train Loss: 0.18359875679016113
Step 10/16000 - Train Loss: 0.20203529298305511
Step 11/16000 - Train Loss: 0.18005189299583435
Step 12/16000 - Train Loss: 0.15201160311698914
Step 13/16000 - Train Loss: 0.17872130870819092
Step 14/16000 - Train Loss: 0.14940166473388672
Step 15/16000 - Train Loss: 0.15894924104213715
Step 16/16000 - Train Loss: 0.17501334846019745
Step 17/16000 - Train Loss: 0.15974003076553345
Step 18/16000 - Train Loss: 0.1736941784620285
Step 19/16000 - Train Loss: 0.1676342934370041
Step 20/16000 - Train Loss: 0.14609622955322266
Step 21/16000 - Train Loss: 0.18225817382335663
Step 

[2026-01-16 17:14:21,441::train::INFO] Iteration 8000 - Train Loss: 0.15166515111923218, Val Loss: 0.15011386293917894


Step 8001/16000 - Train Loss: 0.1402907371520996
Step 8002/16000 - Train Loss: 0.15378296375274658
Step 8003/16000 - Train Loss: 0.1997147500514984
Step 8004/16000 - Train Loss: 0.10956841707229614
Step 8005/16000 - Train Loss: 0.1849341094493866
Step 8006/16000 - Train Loss: 0.14120137691497803
Step 8007/16000 - Train Loss: 0.1238517090678215
Step 8008/16000 - Train Loss: 0.12862978875637054
Step 8009/16000 - Train Loss: 0.09942561388015747
Step 8010/16000 - Train Loss: 0.16178739070892334
Step 8011/16000 - Train Loss: 0.14156058430671692
Step 8012/16000 - Train Loss: 0.11503897607326508
Step 8013/16000 - Train Loss: 0.17928658425807953
Step 8014/16000 - Train Loss: 0.15048402547836304
Step 8015/16000 - Train Loss: 0.1502634882926941
Step 8016/16000 - Train Loss: 0.14914047718048096
Step 8017/16000 - Train Loss: 0.15175959467887878
Step 8018/16000 - Train Loss: 0.1981378048658371
Step 8019/16000 - Train Loss: 0.13038712739944458
Step 8020/16000 - Train Loss: 0.14642350375652313
Step 8

[2026-01-16 17:55:54,003::train::INFO] Iteration 16000 - Train Loss: 0.18241912126541138, Val Loss: 0.15203105813513199


In [7]:
import torch

def mask_tensors(value):
    if isinstance(value, torch.Tensor):
        return f'Tensor with shape {value.shape}'
    elif isinstance(value, dict):
        return {k: mask_tensors(v) for k, v in value.items()}
    elif isinstance(value, list):
        return [mask_tensors(v) for v in value]
    return value

def inspect_pt_file(file_path):
    data = torch.load(file_path, map_location='cpu')
    
    if isinstance(data, dict):
        for key, value in data.items():
            print(f'Key: {key}')
            print(mask_tensors(value))

# Beispielaufruf
file_path = "./AE_all.pt"  # Ersetze mit deinem Dateipfad
inspect_pt_file(file_path)


Key: args
Namespace(beta_1=0.0001, beta_T=0.05, categories=['all'], dataset_dir='./data/ShapeNetCore.v2.PC15k.Resplit', device='cuda', end_lr=0.0001, flexibility=0.0, latent_dim=256, log_root='./logs_ae', logging=True, lr=0.001, max_grad_norm=10, max_iters=1000000, num_inspect_batches=1, num_inspect_pointclouds=4, num_steps=200, num_val_batches=30, num_workers=4, residual=True, resume=None, scale_mode='shape_unit', sched_end_epoch=400000, sched_mode='linear', sched_start_epoch=200000, seed=2020, tag='ALL', train_batch_size=128, val_batch_size=32, val_freq=10000.0, weight_decay=0)
Key: state_dict
{'encoder.conv1.weight': 'Tensor with shape torch.Size([128, 3, 1])', 'encoder.conv1.bias': 'Tensor with shape torch.Size([128])', 'encoder.conv2.weight': 'Tensor with shape torch.Size([128, 128, 1])', 'encoder.conv2.bias': 'Tensor with shape torch.Size([128])', 'encoder.conv3.weight': 'Tensor with shape torch.Size([256, 128, 1])', 'encoder.conv3.bias': 'Tensor with shape torch.Size([256])', 'e

In [4]:
# AE Training with Pytorch Lightning
import torch
import pytorch_lightning as pl
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import os

# Arguments
args = {
    'latent_dim': 256,
    'num_steps': 200,
    'lr': 1e-3,
    'train_batch_size': 128,
    'val_batch_size': 32,
    'categories': ['airplane', 'chair', 'door', 'piano', 'stool', 'toilet', 'vase', 'tent', 'sofa', 'plant'],
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'residual': True,
    'beta_1': 1e-4,
    'beta_T': 0.05,
    'sched_mode': 'linear',
    'max_iters': 70000,  # Maximale Anzahl an Iterationen
    'val_freq': 14000,
    'log_root': './logs_ae',
    'resume': './AE_all.pt'
}

# Setup
seed_all(2020)
logger = get_logger('train', args['log_root'])
writer = SummaryWriter(args['log_root'])

# Datasets and loaders
train_dataset = ModelNet40Dataset(
    root_dir="Dataset/ModelNet40",
    classes=args['categories'],
    split='train',
    normalization="shape_unit",
    target_num_points=NUM_POINTS
)

val_loader = DataLoader(
    ModelNet40Dataset(
        root_dir="Dataset/ModelNet40",
        classes=args['categories'],
        split='test',
        normalization="shape_unit",
        target_num_points=NUM_POINTS
    ),
    batch_size=args['val_batch_size']
)

# Model loading
model = AutoEncoder(args).to(args['device'])
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
start_step = 1

if os.path.exists(args['resume']):
    checkpoint = torch.load(args['resume'], map_location=args['device'])

    if 'state_dict' in checkpoint:
        model.load_state_dict(checkpoint['state_dict'], strict=False)
    elif 'model' in checkpoint:
        model.load_state_dict(checkpoint['model'])
    else:
        raise KeyError("Checkpoint file does not contain expected keys for model state.")

    if 'optimizer_states' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer_states'])
        print("Optimizer state loaded.")
    else:
        print("Optimizer state not found. Initializing new optimizer.")
        optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])

    if 'lr_schedulers' in checkpoint:
        scheduler.load_state_dict(checkpoint['lr_schedulers'])
        print("Scheduler state loaded.")

    start_step = checkpoint.get('global_step', 0) + 1
    print(f"Checkpoint loaded, continuing from step {start_step}")

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# PyTorch Lightning Training
checkpoint_callback = ModelCheckpoint(
    dirpath=args['log_root'],
    filename='AE-{step:06d}-{val_loss:.2f}',
    save_top_k=3,
    monitor='val_loss',
    mode='min'
)

trainer = pl.Trainer(
    max_steps=args['max_iters'],
    val_check_interval=args['val_freq'],
    gpus=1 if torch.cuda.is_available() else 0,
    callbacks=[checkpoint_callback]
)

def train_step(batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0].to(args['device'])
    loss = model.get_loss(x)
    loss.backward()
    optimizer.step()
    return loss

def validate():
    model.eval()
    with torch.no_grad():
        total_loss = sum(model.get_loss(batch[0].to(args['device'])).item() for batch in val_loader)
    return total_loss / len(val_loader)

# Training loop with PyTorch Lightning
train_loader = DataLoader(train_dataset, batch_size=args['train_batch_size'], shuffle=True)

step = start_step
while step <= args['max_iters']:
    for batch in train_loader:
        loss = train_step(batch)
        print(f"Step {step}/{args['max_iters']} - Train Loss: {loss.item()}")

        if step % args['val_freq'] == 0:
            val_loss = validate()
            writer.add_scalar('train/loss', loss.item(), step)
            writer.add_scalar('val/loss', val_loss, step)
            logger.info(f"Step {step} - Train Loss: {loss.item()}, Val Loss: {val_loss}")

            # Manuell speichern im Lightning-Format
            val_checkpoint_path = os.path.join(args['log_root'], f'val_checkpoint_step_{step}.ckpt')
            torch.save({
                'state_dict': model.state_dict(),
                'optimizer_states': optimizer.state_dict(),
                'lr_schedulers': scheduler.state_dict(),
                'global_step': step
            }, val_checkpoint_path)
            
            print(f"Model checkpoint saved at {val_checkpoint_path} during validation")

        scheduler.step()
        step += 1
        if step > args['max_iters']:
            break

print("Training completed!")

final_checkpoint_path = os.path.join(args['log_root'], 'final_model.ckpt')
torch.save({
    'state_dict': model.state_dict(),
    'optimizer_states': optimizer.state_dict(),
    'lr_schedulers': scheduler.state_dict(),
    'global_step': step
}, final_checkpoint_path)

print(f"Final model saved at {final_checkpoint_path}")

final_checkpoint_path = os.path.join(args['log_root'], 'final_model.pt')
torch.save({
    'args': args,  # Include args for reloading during inference
    'state_dict': model.state_dict(),  # Model weights
    'optimizer_states': optimizer.state_dict(),  # Optimizer state for resuming training
    'lr_schedulers': scheduler.state_dict(),  # Learning rate scheduler
    'global_step': step  # Last training step
}, final_checkpoint_path)

print(f"Final model saved at {final_checkpoint_path}")


# Extract latent representations for visualization
def extract_latent_representations(dataloader, model, device):
    model.eval()
    z_representations = []
    labels = []

    with torch.no_grad():
        for batch in dataloader:
            x, label = batch
            x = x.to(device)
            z = model.encode(x)
            z_representations.append(z.cpu().numpy())
            labels.append(label.numpy())

    z_representations = np.concatenate(z_representations, axis=0)
    labels = np.concatenate(labels, axis=0)
    return z_representations, labels

print("Extracting latent representations for visualization...")
classes = args["categories"]
z_vectors, class_labels = extract_latent_representations(val_loader, model, args['device'])

# Apply t-SNE for dimensionality reduction
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, random_state=42)
z_2d = tsne.fit_transform(z_vectors)

# Visualization of t-SNE results with explicit legend
plt.figure(figsize=(8, 6))
scatter = plt.scatter(z_2d[:, 0], z_2d[:, 1], c=class_labels, cmap='viridis', alpha=0.7)
plt.colorbar(scatter, label='Class Labels')

# Create a legend for the class labels
unique_labels = np.unique(class_labels)
legend_handles = [
    plt.Line2D([0], [0], marker='o', color='w',
               markerfacecolor=scatter.cmap(scatter.norm(label)),
               markersize=10, label=classes[int(label)])
    for label in unique_labels
]

plt.legend(handles=legend_handles, title="Categories")
plt.title('t-SNE Visualization of Latent Representations')
plt.xlabel('t-SNE Dim 1')
plt.ylabel('t-SNE Dim 2')
plt.grid(True)
plt.show()


KeyboardInterrupt: 

In [ ]:
# AE Testing

import os
import torch
from tqdm.auto import tqdm
import numpy as np
import time
#import pptk  # Visualisierung von Punktwolken
import threading
import argparse

# Arguments (simplified for testing)
args = {
    'ckpt': './AE_all.pt',  # Pretrained AE checkpoint  ./logs_ae/ckpt_0.004693_100.pt   ./AE_all.pt
    'categories': ['airplane'],
    'device': 'cuda',
    'dataset_path': 'Dataset/ModelNet40',  # ModelNet40 path
    'batch_size': 1,
    'num_steps': 200,
    'save_dir': './results'  # Standardwert für save_dir hinzufügen
}

# Logging
save_dir = os.path.join(args['save_dir'], f"AE_Ours_{'_'.join(args['categories'])}_{int(time.time())}")
os.makedirs(save_dir, exist_ok=True)
logger = get_logger('test', save_dir)


# Loading checkpoint
ckpt = torch.load(args['ckpt'], map_location=torch.device(args['device']))
if isinstance(ckpt['args'], argparse.Namespace):
    ckpt['args'] = vars(ckpt['args'])

# Ensure num_steps matches between checkpoint and current configuration
if 'num_steps' not in args or args['num_steps'] != ckpt['args']['num_steps']:
    print(f"Updating num_steps to match checkpoint: {ckpt['args']['num_steps']}")
    args['num_steps'] = ckpt['args']['num_steps']

# Reinitializing the diffusion scheduler to match the new num_steps
if args['num_steps'] != len(ckpt['state_dict']['diffusion.var_sched.betas']):
    print(f"Reinitializing diffusion scheduler to match num_steps: {args['num_steps']}")
    
    # Define appropriate beta_1 and beta_T values (you can adjust them based on your use case)
    beta_1 = ckpt['args'].get('beta_1', 1e-4)  # Default or checkpoint value
    beta_T = ckpt['args'].get('beta_T', 0.02)  # Default or checkpoint value

    # Initialize the VarianceSchedule with these parameters
    diffusion_scheduler = VarianceSchedule(num_steps=args['num_steps'], beta_1=beta_1, beta_T=beta_T)

    # Update the state_dict of the checkpoint with the new scheduler
    ckpt['state_dict']['diffusion.var_sched.betas'] = diffusion_scheduler.betas
    ckpt['state_dict']['diffusion.var_sched.alphas'] = diffusion_scheduler.alphas
    ckpt['state_dict']['diffusion.var_sched.alpha_bars'] = diffusion_scheduler.alpha_bars
    ckpt['state_dict']['diffusion.var_sched.sigmas_flex'] = diffusion_scheduler.sigmas_flex
    ckpt['state_dict']['diffusion.var_sched.sigmas_inflex'] = diffusion_scheduler.sigmas_inflex


# Load model
model = AutoEncoder(ckpt['args']).to(args['device'])

# Load state_dict with strict mode
try:
    model.load_state_dict(ckpt['state_dict'])
    print("Model state loaded successfully.")
except RuntimeError as e:
    print(f"Error loading model state_dict: {e}")


# Load dataset
test_dset = ModelNet40Dataset(root_dir=args['dataset_path'], classes=args['categories'], normalization="shape_unit", target_num_points=NUM_POINTS)
test_loader = DataLoader(test_dset, batch_size=args['batch_size'], num_workers=0)

# Downsampling-Funktion
def downsample_points(points, target_num_points=2048):
    num_points = points.size(1)  # M
    if num_points > target_num_points:
        indices = torch.randperm(num_points)[:target_num_points]  # Zufällige Auswahl
        points = points[:, indices]  # Auswahl der Punkte
    elif num_points < target_num_points:
        padding = torch.zeros(points.size(0), target_num_points - num_points, 3).to(points.device)  # Padding
        points = torch.cat((points, padding), dim=1)
    return points

# Auto-close function for viewers after timeout
def close_viewer_after_timeout(viewer, timeout=10):
    # Close viewer after the specified timeout
    time.sleep(timeout)
    viewer.close()
    print(f"Viewer closed after {timeout} seconds.")

# Define visualization points
num_batches = len(test_loader)
visualize_at = {num_batches - 1}

# Testing loop
all_ref, all_recons = [], []

features_real, features_fake = [], []

model.eval()

for batch_idx, batch in enumerate(tqdm(test_loader, total=num_batches)):
    ref_batch = batch[0].to(args['device'])  # Get pointcloud
    
    for i in range(ref_batch.size(0)):  # Loop over each point cloud in the batch
        ref = ref_batch[i:i+1, :]  # Get the individual point cloud (batch size 1)
        
        ref = downsample_points(ref, target_num_points=2048)

        with torch.no_grad():
            code_real = model.encode(ref)  # Extract feature for real point cloud
            recons = model.decode(code_real, ref.size(1)).detach()
            code_fake = model.encode(recons)  # Extract feature for generated point cloud

        features_real.append(code_real.cpu().numpy())
        features_fake.append(code_fake.cpu().numpy())

        if len(batch) > 2:
            shift, scale = batch[1].to(args['device'])[i:i+1], batch[2].to(args['device'])[i:i+1]
        else:
            shift, scale = torch.zeros_like(ref), torch.ones_like(ref)

        with torch.no_grad():
            code = model.encode(ref)
            recons = model.decode(code, ref.size(1)).detach()

        recons = recons * scale + shift
        
        all_ref.append(ref.cpu())
        all_recons.append(recons.cpu())

        if batch_idx in visualize_at:
            print(f"Visualizing batch {batch_idx}, sample {i}...")
            
            ref_points = ref.cpu().numpy().reshape(-1, 3)
            recons_points = recons.cpu().numpy().reshape(-1, 3)

            # Reference cloud viewer
            #print(f"Initializing reference viewer for batch {batch_idx}, sample {i}...")
            #v1 = pptk.viewer(ref_points)
            #v1.set(point_size=0.01)
            #print("Reference viewer initialized.")

            # Reconstructed cloud viewer
            #print(f"Initializing reconstruction viewer for batch {batch_idx}, sample {i}...")
            #v2 = pptk.viewer(recons_points)
            #v2.set(point_size=0.01)
            #print("Reconstruction viewer initialized.")

            # PyVista visualization for reference and reconstructed clouds
            print(f"Visualizing reference cloud with Open3D...")
            visualize_with_open3d(ref_points, title="Reference Point Cloud")

            print(f"Visualizing reconstructed cloud with Open3D...")
            visualize_with_open3d(recons_points, title="Reconstructed Point Cloud")

            # Set a timer to auto-close viewers after 10 seconds
            #threading.Thread(target=close_viewer_after_timeout, args=(v1, 10)).start()
            #threading.Thread(target=close_viewer_after_timeout, args=(v2, 10)).start()

# Convert lists to numpy arrays
features_real = np.concatenate(features_real, axis=0)
features_fake = np.concatenate(features_fake, axis=0)

# Compute FID score
fid_score = calculate_fid(features_real, features_fake)
logger.info(f"FID Score: {fid_score:.6f}")

# Compute metrics
metrics = chamfer(torch.cat(all_recons).to(args['device']), torch.cat(all_ref).to(args['device']), batch_size=args['batch_size'])
logger.info(f"CD: {metrics['MMD-CD'].item():.12f}")


C:\Users\Sirius\AppData\Local\Temp\ipykernel_19860\1863768767.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(args['ckpt'], map_location=torch.device(

Reinitializing diffusion scheduler to match num_steps: 200
Model state loaded successfully.


  0%|          | 0/209 [00:00<?, ?it/s]

Visualizing batch 208, sample 0...
Visualizing reference cloud with Open3D...
Visualizing reconstructed cloud with Open3D...
Visualizing batch 208, sample 1...
Visualizing reference cloud with Open3D...
Visualizing reconstructed cloud with Open3D...


[2026-01-16 18:17:44,443::test::INFO] FID Score: 0.245187


EMD-CD:   0%|          | 0/209 [00:00<?, ?it/s]

[2026-01-16 18:17:45,158::test::INFO] CD: 0.016268962994
